In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score,recall_score,precision_score, confusion_matrix
from sklearn.metrics import silhouette_score
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from dateutil.relativedelta import relativedelta
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel

# Data preparation

In [2]:
#this dataframe is generated from "Customer_Segmentation_and_Data_Merge_Ab4J" notebook
df=pd.read_csv('customer_revenue_tier_price.csv').drop('Unnamed: 0',axis=1)

#the timeframe of this data is too short for feature engineering
#cx_case=pd.read_csv('cx_cases.csv') 

#this is from [USC_IB_Churn_Analysis_DB].[dbo].[VJ_CX_Interactions_Tab]
cx_interact=pd.read_csv('cx_interactions_0625.csv')
#this is from [USC_IB_Churn_Analysis_DB].[dbo].[VJ_Sites_Install_Base_Tab]
site_install=pd.read_csv('site_install_0722.csv')
#this is from [USC_IB_Churn_Analysis_DB].[dbo].[VJ_SVC_Incidents_Tab]
svc_incident = pd.read_csv('svc_incidents_0625.csv')

In [126]:
#check if the data shape is the same as generated from the "Customer_Segmentation_and_Data_Merge_Ab4J" notebook
df.shape

(602386, 64)

In [4]:
def multiple_item(x):
    #if customer used certain Sales Channel or purchase certain Product mode equally frequently
    #this function help to return the value as "Multiple", which would be helpful when creating 
    #dummy variable later
    if len(x[0])>1:
        return "Multiple"
    else:
        return x

In [5]:
def LatestAmtRatio(customer_id,duration,trx_max,frequency,monetary,data_end):
    #compare if customer spend more in latest 1/3 period
    #if yes, the ratio would be larger than 1
    if frequency <= 2:
        return 1
    else:
        if duration <= 366:
            return 1
        else:
            k = round(duration/3)
            delta=pd.to_timedelta('{} days'.format(k))
            cal_start_date = trx_max-delta
            ## using the df
            latest_amt = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                            (df['TRX_DATE']<=data_end)&\
                            (df['TRX_DATE']>=cal_start_date)]['TRX_AMT_USD'].sum()
            latest_ratio = (latest_amt*3)/monetary
            return latest_ratio

In [6]:
def LatestPriceRatioTrend(customer_id,duration,trx_max,frequency,agg_price_index,data_end):
    #compare if customer tend to have higher price index in recent 1/3 period
    #if yes, the ratio would be larger than 1
    if frequency <= 2:
        return 1
    else:
        if duration <= 366:
            return 1
        else:
            k = round(duration/3)
            delta=pd.to_timedelta('{} days'.format(k))
            cal_start_date = trx_max-delta
            ## using the df
            latest_ws = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                           (df['TRX_DATE']<=data_end)&\
                           (df['TRX_DATE']>=cal_start_date)]['Price_WS_proportion'].sum()
            latest_stu = df[(df['CUSTOMER_SITE_ID']==customer_id)&\
                            (df['TRX_DATE']<=data_end)&\
                            (df['TRX_DATE']>=cal_start_date)]['Price_STU_proportion'].sum()
            latest_price_ratio= latest_ws/latest_stu
            latest_price_ratio_trend = latest_price_ratio/agg_price_index
            return latest_price_ratio_trend

In [7]:
def DfTimeFeature(df,data_start,data_end,svc_incident,site_install,cx_interact):
    df['TRX_DATE']=pd.to_datetime(df['TRX_DATE'])
    data_start =pd.to_datetime(data_start)
    data_end =pd.to_datetime(data_end)
    
    #categorize the product family for later dummy variable creation
    df['Main_Product']=np.where(df['PRODUCT_FAMILY'].isin(['CIJ','LCM','TTO','TIJ']),df['PRODUCT_FAMILY'],'Other')
    
    #for each transaction, adjust the weight to calculate overall/aggregate price index later
    df['Qty_proportion']=df['QUANTITY']/df['Qty']
    df['Price_WS_proportion']=df['Site_Level_Price_Index_WS']*df['Qty_proportion']
    df['Price_STU_proportion']=df['Site_Level_Price_Index_STU']*df['Qty_proportion']
    
    #set the timeframe for feature calculation
    df_time = df[(df['TRX_DATE']>=data_start)&(df['TRX_DATE']<=data_end)]
    

    #aggrefate on customer_site_id level
    dfg=df_time.groupby(['CUSTOMER_SITE_ID','Tier',\
                      'SHORT_VERTICAL','CUSTOMER_CLASS'])\
    .agg({"SALES_CHANNEL":pd.Series.mode,'PRODUCT_MODEL':pd.Series.mode,'Main_Product':pd.Series.mode,\
     'TRX_DATE':['min','max'],'TRX_AMT_USD':'sum','CUSTOMER_TRX_ID':pd.Series.nunique,\
        "ITEM_ID":pd.Series.nunique,'Price_WS_proportion':'sum','Price_STU_proportion':'sum'}).reset_index()

    dfg.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in dfg.columns]
    
    #get feature from the join df
    dfg=dfg.rename(columns={'TRX_AMT_USD_sum':'Monetary','CUSTOMER_TRX_ID_nunique':'Frequency',\
                     'ITEM_ID_nunique':'Unique_Item_Purchased'})
    dfg['Aggregate_index']=dfg['Price_WS_proportion_sum']/dfg['Price_STU_proportion_sum']
    dfg['Duration']=((dfg['TRX_DATE_max']-dfg['TRX_DATE_min'])/np.timedelta64(1,'D')).astype('int')+1
    dfg['Recency']=(((data_end-dfg['TRX_DATE_max'])/np.timedelta64(1,'D')).astype('int')+1)
    dfg['SALES_CHANNEL_mode']=dfg['SALES_CHANNEL_mode'].apply(multiple_item)
    dfg['PRODUCT_MODEL_mode']=dfg['PRODUCT_MODEL_mode'].apply(multiple_item)
    dfg['Main_Product_mode']=dfg['Main_Product_mode'].apply(multiple_item)
    dfg['Avg_AMT']=dfg['Monetary']/dfg['Frequency']
    dfg['Avg_Duration']=dfg['Duration']/dfg['Frequency']
    dfg_with_dummy = pd.get_dummies(dfg, columns=['SHORT_VERTICAL', 'CUSTOMER_CLASS',
                                               'SALES_CHANNEL_mode', 'PRODUCT_MODEL_mode','Main_Product_mode'])
    dfg_with_dummy['Lastest_Amt_Ratio']=dfg_with_dummy.apply(lambda x: LatestAmtRatio(x.CUSTOMER_SITE_ID, \
                                     x.Duration,x.TRX_DATE_max,x.Frequency,x.Monetary,data_end), axis=1)
    dfg_with_dummy['Lastest_Price_Ratio_Trend']=dfg_with_dummy.apply(lambda x: LatestPriceRatioTrend(x.CUSTOMER_SITE_ID, \
                                     x.Duration,x.TRX_DATE_max,x.Frequency,x.Aggregate_index,data_end), axis=1)
    
    #get feature from svc_incident
    svc_incident['LAST_SERVICE_EVENT_DATE']=pd.to_datetime(svc_incident['LAST_SERVICE_EVENT_DATE'])
    svc_time = svc_incident[(svc_incident['LAST_SERVICE_EVENT_DATE']>=data_start)&\
                               (svc_incident['LAST_SERVICE_EVENT_DATE']<=data_end)]
    incident_feature=svc_time.groupby('CUSTOMER_SITE_ID').agg({'NO_OF_SVC_INCIDENTS':'sum'}).reset_index()
    
    site_install['INSTANCE_LAST_UPDATE_DATE']=pd.to_datetime(site_install['INSTANCE_LAST_UPDATE_DATE'])
    site_time = site_install[(site_install['INSTANCE_LAST_UPDATE_DATE']<=data_end)]
    site_time['Contract']=np.where(site_time['CONTRACT_FLAG']=='Y',1,0)
    site_feature=site_time.groupby('CUSTOMER_SITE_ID').agg({"INSTANCE_ID":pd.Series.nunique,\
                                                           "Contract":'sum'}).reset_index()
    
    id_with_incident=dfg_with_dummy.merge(incident_feature,on='CUSTOMER_SITE_ID')['CUSTOMER_SITE_ID'].values
    df_no_incident=dfg_with_dummy[~(dfg_with_dummy['CUSTOMER_SITE_ID'].isin(id_with_incident))]
    df_no_incident['NO_OF_SVC_INCIDENTS']=0
    #some customer don't have incident but stil have contracted printer
    df_no_incident2=df_no_incident.merge(site_feature[['CUSTOMER_SITE_ID','Contract']],on='CUSTOMER_SITE_ID',how='left')
    df_no_incident2['Contract']=df_no_incident2['Contract'].fillna(0)
    df_no_incident2['Incident_per_printer']=0
    
    df_incident=dfg_with_dummy.merge(incident_feature,on='CUSTOMER_SITE_ID').merge(site_feature,on='CUSTOMER_SITE_ID')
    df_incident['Incident_per_printer']=df_incident['NO_OF_SVC_INCIDENTS']/df_incident['INSTANCE_ID']
    df_incident=df_incident.drop('INSTANCE_ID',axis=1)
    
    df_feature=pd.concat([df_incident,df_no_incident2])
    
    #get feature from cx interact
    cx_interact['Created_Date']=pd.to_datetime(cx_interact['Created_Date'])
    cx_time= cx_interact[(cx_interact['Created_Date']>=data_start)&\
                        (cx_interact['Created_Date']<=data_end)]
    interact_feature=cx_time.groupby('CUSTOMER_SITE_ID').agg({'Visit':'sum'}).reset_index()
    df_feature2=df_feature.merge(interact_feature,on='CUSTOMER_SITE_ID',how='left')
    df_feature2['Visit']=df_feature2['Visit'].fillna(0)
    df_feature2['Visit_per_TRX']=df_feature2['Visit']/df_feature2['Frequency']
    
    return df_feature2

In [8]:
data_start_18='2015-01-01'
data_end_18='2018-06-01'
first_df=DfTimeFeature(df,data_start_18,data_end_18,svc_incident,site_install,cx_interact)

start_time = pd.to_datetime('2018-06-01')
end_time = pd.to_datetime('2019-06-01')
active_customer=df[(df['TRX_DATE']>start_time)&(df['TRX_DATE']<=end_time)]['CUSTOMER_SITE_ID'].values
first_df["Churn"]=np.where(first_df['CUSTOMER_SITE_ID'].isin(active_customer),0,1)

In [9]:
data_start_19='2015-01-01'
data_end_19='2019-06-01'
second_df=DfTimeFeature(df,data_start_19,data_end_19,svc_incident,site_install,cx_interact)

start_time = pd.to_datetime('2019-06-01')
end_time = pd.to_datetime('2020-06-03')
active_customer=df[(df['TRX_DATE']>start_time)&(df['TRX_DATE']<=end_time)]['CUSTOMER_SITE_ID'].values
second_df["Churn"]=np.where(second_df['CUSTOMER_SITE_ID'].isin(active_customer),0,1)

In [10]:
data_start='2015-01-01'
data_end='2020-06-03'

third_df=DfTimeFeature(df,data_start,data_end,svc_incident,site_install,cx_interact)

In [11]:
#large number of dummy variable created are NOT included for calculation efficiency and model intepretation purpose

all_features=['Frequency','Recency','Unique_Item_Purchased', 'Aggregate_index',
            'PRODUCT_MODEL_mode_Multiple', 'PRODUCT_MODEL_mode_MAKE-UP','PRODUCT_MODEL_mode_SOLVENT',
            'SALES_CHANNEL_mode_Esker','SHORT_VERTICAL_UNKNOWN',
            'Main_Product_mode_CIJ', 'Main_Product_mode_LCM','Main_Product_mode_TIJ', 'Main_Product_mode_TTO',
            'CUSTOMER_CLASS_DISTRIBUTOR', 'CUSTOMER_CLASS_OEM', 
            'NO_OF_SVC_INCIDENTS', 'Contract','Lastest_Amt_Ratio','Lastest_Price_Ratio_Trend',
            'Incident_per_printer', 'Visit_per_TRX','Avg_AMT','Avg_Duration']

In [12]:
#Tier 4 customer are not included in model testing part, since they are wither one-time buyer or new customer
#so, it is not meaningful to build churn model for them

tier1_18 = first_df[first_df['Tier']=='Tier 1']
tier2_18 = first_df[first_df['Tier']=='Tier 2']
tier3_18 = first_df[first_df['Tier']=='Tier 3']
#tier4_18 = first_df[first_df['Tier']=='Tier 4']

X1= tier1_18.drop(['Tier',"Churn"],axis=1)[all_features]
X2= tier2_18.drop(['Tier',"Churn"],axis=1)[all_features]
X3= tier3_18.drop(['Tier',"Churn"],axis=1)[all_features]
#X4= tier4_18.drop(['Tier',"Churn"],axis=1)

y1=tier1_18["Churn"]
y2=tier2_18["Churn"]
y3=tier3_18["Churn"]
#y4=tier4_18["Churn"]

In [13]:
tier1_19 = second_df[second_df['Tier']=='Tier 1']
tier2_19 = second_df[second_df['Tier']=='Tier 2']
tier3_19 = second_df[second_df['Tier']=='Tier 3']
#tier4_19 = second_df[second_df['Tier']=='Tier 4']

X1_19= tier1_19.drop(['Tier',"Churn"],axis=1)[all_features]
X2_19= tier2_19.drop(['Tier',"Churn"],axis=1)[all_features]
X3_19= tier3_19.drop(['Tier',"Churn"],axis=1)[all_features]
#X4_19= tier4_19.drop(['Tier',"Churn"],axis=1)

y1_19=tier1_19["Churn"]
y2_19=tier2_19["Churn"]
y3_19=tier3_19["Churn"]
#y4_19=tier4_19["Churn"]

In [14]:
tier1_20 = third_df[third_df['Tier']=='Tier 1']
tier2_20 = third_df[third_df['Tier']=='Tier 2']
tier3_20 = third_df[third_df['Tier']=='Tier 3']
#tier4_20 = third_phase[third_phase['Tier']=='Tier 4']

X1_20= tier1_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
X2_20= tier2_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
X3_20= tier3_20.drop(['Tier'],axis=1)[['CUSTOMER_SITE_ID']+all_features]
#X4_20= tier4_20.drop(['Tier'],axis=1)

## Model Testing

In [15]:
def LogicResult_allcoef(X,y,Penalty='l2',c=1,Solver='lbfgs'):
    
    #split data
    X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, \
                                                        stratify=y, random_state=0)
    #scale features
    scaler_train=StandardScaler()
    scaler_train.fit(X_train)
    X_train_scaled = pd.DataFrame(scaler_train.transform(X_train),columns = X_train.columns)
    
    scaler_test=StandardScaler()
    scaler_test.fit(X_test)
    X_test_scaled = pd.DataFrame(scaler_test.transform(X_test),columns = X_test.columns)
    
    #Buidle logistic regression model
    classifier = LogisticRegression(random_state=0, penalty=Penalty,solver=Solver,C=c)
    classifier.fit(X_train_scaled, y_train)
    
    #coefficient table
    coef_table = pd.DataFrame(list(X_train.columns)).copy()
    coef_table.insert(len(coef_table.columns),"Coefs",classifier.coef_.transpose())
    coef_table.columns=['Variable','Coefs']
    coef_table['Coefs_ABS']=coef_table['Coefs'].abs()
    coef=coef_table.sort_values('Coefs_ABS',ascending=False).reset_index(drop=True)
    coef=coef.drop('Coefs_ABS',axis=1)

    
    y_pred = classifier.predict(X_test_scaled)
    acc = accuracy_score(y_test,y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test,y_pred)
    results = pd.DataFrame([['Logistic Regression', acc,prec,rec,f1]],\
                       columns=['Model', 'Accuracy', 'Precision', 'Recall','F1 Score'])
    return coef,results

In [16]:
def LogicResult_allcoef_10fold(X,y,Penalty,c,Solver):
    #10 fold cross validation on logistic regression
    scoring = ['accuracy','precision', 'recall','f1']

    classifier = LogisticRegression(random_state=0, penalty=Penalty,solver=Solver,C=c)
    clf = make_pipeline(preprocessing.StandardScaler(), classifier)
    scores = cross_validate(clf, X, y, scoring=scoring,cv=10)

    return scores

In [17]:
pen=['l1','l2']
c = [0.01,0.1, 1, 10, 100, 1000]
solver = ['liblinear', 'saga']
def best_Model(X,y,penalty,C,Solver,hyper_param=False):
    #find the nest hyper-parametter or view the highest 10-fold average model metric result
    rank = pd.DataFrame(columns = ['accuracy','precision','recall','f1'])
    for i in penalty:
        for j in C:
            for k in Solver:
                tmp_score = LogicResult_allcoef_10fold(X,y,i,j,k)
                tmp_name = 'Logistic {i1} C={j1} {k1}'.format(i1=i,j1=j,k1=k)
                rank.loc[tmp_name,'accuracy'] = tmp_score['test_accuracy'].mean()
                rank.loc[tmp_name,'precision'] = tmp_score['test_precision'].mean()
                rank.loc[tmp_name,'recall'] = tmp_score['test_recall'].mean()
                rank.loc[tmp_name,'f1'] = tmp_score['test_f1'].mean()
                rank.loc[tmp_name,'penalty']=i
                rank.loc[tmp_name,'c']=j
                rank.loc[tmp_name,'solver']=k
    #if all score are the same, prefer lasso to ridge            
    rank = rank.sort_values(['recall','accuracy','f1','penalty'],ascending = False)
    if hyper_param == False:
        return rank
    else:
        best_pen =rank.head(1)['penalty'].values[0]
        best_c =rank.head(1)['c'].values[0]
        best_sol =rank.head(1)['solver'].values[0]
        return best_pen, best_c, best_sol

In [18]:
def LogicResult_allcoef_next_year(X_cur,y_cur,X_next,y_next,Penalty,c,Solver):
    #model build on previous period but test on next period
    #scale the data
    scaler_current=StandardScaler()
    scaler_current.fit(X_cur)
    X_cur_scaled = pd.DataFrame(scaler_current.transform(X_cur),columns = X_cur.columns)
    
    scaler_next=StandardScaler()
    scaler_next.fit(X_next)
    X_next_scaled = pd.DataFrame(scaler_next.transform(X_next),columns = X_next.columns)
    
    #Buidle logistic regression model
    classifier = LogisticRegression(random_state=0, penalty=Penalty,solver=Solver,C=c)
    classifier.fit(X_cur_scaled, y_cur)
    
    #coefficient table
    coef_table = pd.DataFrame(list(X_cur.columns)).copy()
    coef_table.insert(len(coef_table.columns),"Coefs",classifier.coef_.transpose())
    coef_table.columns=['Variable','Coefs']
    coef_table['Coefs_ABS']=coef_table['Coefs'].abs()
    coef=coef_table.sort_values('Coefs_ABS',ascending=False).reset_index(drop=True)
    coef=coef.drop('Coefs_ABS',axis=1)

    
    y_pred = classifier.predict(X_next_scaled)
    acc = accuracy_score(y_next,y_pred)
    prec = precision_score(y_next, y_pred)
    rec = recall_score(y_next, y_pred)
    f1 = f1_score(y_next,y_pred)
    results = pd.DataFrame([['Logistic Regression', acc,prec,rec,f1]],\
                       columns=['Model', 'Accuracy', 'Precision', 'Recall','F1 Score'])
    return coef,results

In [19]:
def SFM_RFE_featrue_selection(X,y,fit_all=False):
    #use select from model (SFM) and recursive feature elimination (RFE) to select model
    #when testing the model, we don't fit all data
    #but when predicting, we fit all data available
    if fit_all == False:
        
        #split data
        X_train,X_test,y_train,y_test= train_test_split(X,y,test_size=0.2, \
                                                        stratify=y, random_state=0)
        
        #scale data
        scaler_train=StandardScaler()
        scaler_train.fit(X_train)
        X_train_scaled = pd.DataFrame(scaler_train.transform(X_train),columns = X_train.columns)

    
        #SFM
        selector_sfm = SelectFromModel(estimator=LogisticRegression()).fit(X_train_scaled, y_train)
        sfm_feature=X_train.columns[selector_sfm.get_support()]
    
        #RFE
        selector_rfe = RFE(estimator=LogisticRegression(), n_features_to_select=len(sfm_feature), step=1)\
        .fit(X_train_scaled, y_train)
        rfe_feature=X_train.columns[selector_rfe.get_support()]
    
        selected_feature = [x for x in sfm_feature if x in rfe_feature]
        if len(selected_feature)< 5:
            selected_feature_long=rfe_feature.union(sfm_feature)
            return selected_feature_long
        else:
            return selected_feature
    else:
        #scale data
        scaler_train=StandardScaler()
        scaler_train.fit(X)
        X_scaled = pd.DataFrame(scaler_train.transform(X),columns = X.columns)
        
        #SFM
        selector_sfm = SelectFromModel(estimator=LogisticRegression()).fit(X_scaled, y)
        sfm_feature=X.columns[selector_sfm.get_support()]
    
        #RFE
        selector_rfe = RFE(estimator=LogisticRegression(), n_features_to_select=len(sfm_feature), step=1)\
        .fit(X_scaled, y)
        rfe_feature=X.columns[selector_rfe.get_support()]
    
        selected_feature = [x for x in sfm_feature if x in rfe_feature]
        if len(selected_feature)< 5:
            #if the number of features selected by both SFM and RFE is less than 5, use union result instead
            selected_feature_long=rfe_feature.union(sfm_feature)
            return selected_feature_long
        else:
            return selected_feature

# Feature Reduction Result (Model Testing)

## 3 types of Model
Type 1: <br>
Model based on 2015-2019 data/ 80% training and 20 testing <br>
Target period :2018/6 - 2019/6 <br>
Feature period: 2015/1- 2018/6 <br>
<br>
Type 2:<br>
Model based on 2015-2019, but test on 2015-2020 <br>
<br>
Type 3 <br>
Model based on 2015-2020 data/ 80% training and 20 testing <br>
Target period :2019/6 - 2020/6 <br>
Feature period: 2015/1- 2019/6 <br>

## Tier 1

### Type 1 Model

In [20]:
#use all feature
rank1_18_all= best_Model(X1,y1,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X1,y1,pen,c,solver,hyper_param=True)
coef1_18_all,evaluation1_18_all=LogicResult_allcoef(X1,y1,best_pen,best_c,best_sol)
#model metric based on random_state=0 train-test data split
evaluation1_18_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.965152,0.714286,0.192308,0.30303


In [21]:
#10 cross validation result using best hyper parameter for all features
rank1_18_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=1 liblinear,0.965152,0.660931,0.3,0.394294,l2,1.0,liblinear


In [22]:
#use selected feature
sfm_rfe_18_1=SFM_RFE_featrue_selection(X1,y1)
rank1_18= best_Model(X1[sfm_rfe_18_1],y1,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X1[sfm_rfe_18_1],y1,pen,c,solver,hyper_param=True)
coef1_18,evaluation1_18=LogicResult_allcoef(X1[sfm_rfe_18_1],y1,best_pen,best_c,best_sol)
#model metric based on random_state=0 train-test data split
evaluation1_18

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.965152,0.666667,0.230769,0.342857


In [23]:
#10 cross validation result using best hyper parameter for selected features
rank1_18.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=1 saga,0.964848,0.659524,0.276923,0.376112,l2,1.0,saga


### Type 2 Model

In [24]:
#use all feature
best_pen,best_c,best_sol = best_Model(X1,y1,pen,c,solver,hyper_param=True)
coef1_18_next_all,evaluation1_18_next_all=LogicResult_allcoef_next_year(X1,y1,X1_19\
                                                      ,y1_19,best_pen,best_c,best_sol)
evaluation1_18_next_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.939701,0.866667,0.252918,0.391566


In [25]:
#use selected feature
sfm_rfe_18_1_next=SFM_RFE_featrue_selection(X1,y1,fit_all=True)
best_pen,best_c,best_sol = best_Model(X1[sfm_rfe_18_1_next],y1,pen,c,solver,hyper_param=True)
coef1_18_next,evaluation1_18_next=LogicResult_allcoef_next_year(X1[sfm_rfe_18_1_next],y1,X1_19[sfm_rfe_18_1_next]\
                                                      ,y1_19,best_pen,best_c,best_sol)
evaluation1_18_next

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.939403,0.885714,0.241245,0.379205


### Type 3 Model

In [26]:
#use all feature
rank1_19_all= best_Model(X1_19,y1_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X1_19,y1_19,pen,c,solver,hyper_param=True)
coef1_19_all,evaluation1_19_all=LogicResult_allcoef(X1_19,y1_19,best_pen,best_c,best_sol)
evaluation1_19_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.962687,0.825,0.647059,0.725275


In [27]:
#10 cross validation result using best hyper parameter for all features
rank1_19_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l1 C=1 liblinear,0.957612,0.836026,0.560308,0.667848,l1,1.0,liblinear


In [28]:
#use selected feature
sfm_rfe_19_1=SFM_RFE_featrue_selection(X1_19,y1_19)
rank1_19= best_Model(X1_19[sfm_rfe_19_1],y1_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X1_19[sfm_rfe_19_1],y1_19,pen,c,solver,hyper_param=True)
coef1_19,evaluation1_19=LogicResult_allcoef(X1_19[sfm_rfe_19_1],y1_19,best_pen,best_c,best_sol)
evaluation1_19

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.965672,0.888889,0.627451,0.735632


In [29]:
#10 cross validation result using best hyper parameter for selected features
rank1_19.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=1 liblinear,0.959403,0.862805,0.560154,0.677214,l2,1.0,liblinear


In [30]:
#feature selected by Type 1 model for Tier 1
sfm_rfe_18_1

['Recency',
 'Aggregate_index',
 'PRODUCT_MODEL_mode_Multiple',
 'Main_Product_mode_CIJ',
 'Main_Product_mode_TTO',
 'Lastest_Amt_Ratio',
 'Visit_per_TRX']

In [31]:
#feature selected by Type 2 model for Tier 1
sfm_rfe_18_1_next

['Recency',
 'Aggregate_index',
 'PRODUCT_MODEL_mode_Multiple',
 'Main_Product_mode_CIJ',
 'Main_Product_mode_TTO',
 'Contract',
 'Lastest_Amt_Ratio',
 'Visit_per_TRX']

In [32]:
#feature selected by Type 3 model for Tier 1
sfm_rfe_19_1

Index(['CUSTOMER_CLASS_OEM', 'Contract', 'Lastest_Amt_Ratio',
       'Main_Product_mode_TTO', 'NO_OF_SVC_INCIDENTS', 'Recency',
       'SALES_CHANNEL_mode_Esker', 'Visit_per_TRX'],
      dtype='object')

## Tier 2

### Type 1 Model

In [33]:
#use all feature
rank2_18_all= best_Model(X2,y2,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X2,y2,pen,c,solver,hyper_param=True)
coef2_18_all,evaluation2_18_all=LogicResult_allcoef(X2,y2,best_pen,best_c,best_sol)
evaluation2_18_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.889485,0.77907,0.443709,0.565401


In [34]:
#10 cross validation result using best hyper parameter for all features
rank2_18_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=1 liblinear,0.893541,0.790747,0.465316,0.57708,l2,1.0,liblinear


In [35]:
#use selected feature
sfm_rfe_18_2=SFM_RFE_featrue_selection(X2,y2)
rank2_18= best_Model(X2[sfm_rfe_18_2],y2,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X2[sfm_rfe_18_2],y2,pen,c,solver,hyper_param=True)
coef2_18,evaluation2_18=LogicResult_allcoef(X2[sfm_rfe_18_2],y2,best_pen,best_c,best_sol)
evaluation2_18

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.888412,0.783133,0.430464,0.555556


In [36]:
#10 cross validation result using best hyper parameter for selected features
rank2_18.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=0.1 liblinear,0.896331,0.812294,0.465368,0.585322,l2,0.1,liblinear


### Type 2 Model

In [37]:
#use all feature
best_pen,best_c,best_sol = best_Model(X2,y2,pen,c,solver,hyper_param=True)
coef2_18_next_all,evaluation2_18_next_all=LogicResult_allcoef_next_year(X2,y2,X2_19\
                                                      ,y2_19,best_pen,best_c,best_sol)
evaluation2_18_next_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.862921,0.848987,0.432052,0.572671


In [38]:
#use selected feature
sfm_rfe_18_2_next=SFM_RFE_featrue_selection(X2,y2,fit_all=True)
best_pen,best_c,best_sol = best_Model(X2[sfm_rfe_18_2_next],y2,pen,c,solver,hyper_param=True)
coef2_18_next,evaluation2_18_next=LogicResult_allcoef_next_year(X2[sfm_rfe_18_2_next],y2,X2_19[sfm_rfe_18_2_next]\
                                                      ,y2_19,best_pen,best_c,best_sol)
evaluation2_18_next

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.862323,0.848148,0.429241,0.570006


### Type 3 Model

In [39]:
#use all feature
rank2_19_all= best_Model(X2_19,y2_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X2_19,y2_19,pen,c,solver,hyper_param=True)
coef2_19_all,evaluation2_19_all=LogicResult_allcoef(X2_19,y2_19,best_pen,best_c,best_sol)
evaluation2_19_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.871514,0.823077,0.502347,0.623907


In [40]:
#10 cross validation result using best hyper parameter for all features
rank2_19_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=100 saga,0.878461,0.829577,0.551129,0.653221,l2,100.0,saga


In [41]:
#use selected feature
sfm_rfe_19_2=SFM_RFE_featrue_selection(X2_19,y2_19)
rank2_19= best_Model(X2_19[sfm_rfe_19_2],y2_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X2_19[sfm_rfe_19_2],y2_19,pen,c,solver,hyper_param=True)
coef2_19,evaluation2_19=LogicResult_allcoef(X2_19[sfm_rfe_19_2],y2_19,best_pen,best_c,best_sol)
evaluation2_19

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.86753,0.822581,0.478873,0.605341


In [42]:
#10 cross validation result using best hyper parameter for selected features
rank2_19.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=10 liblinear,0.880254,0.844537,0.546429,0.654731,l2,10.0,liblinear


In [43]:
#feature selected by Type 1 model for Tier 2
sfm_rfe_18_2

['Frequency',
 'Recency',
 'PRODUCT_MODEL_mode_Multiple',
 'NO_OF_SVC_INCIDENTS',
 'Avg_AMT']

In [44]:
#feature selected by Type 2 model for Tier 2
sfm_rfe_18_2_next

Index(['Avg_AMT', 'Frequency', 'Main_Product_mode_CIJ',
       'Main_Product_mode_TIJ', 'Main_Product_mode_TTO', 'NO_OF_SVC_INCIDENTS',
       'PRODUCT_MODEL_mode_Multiple', 'Recency'],
      dtype='object')

In [45]:
#feature selected by Type 3 model for Tier 2
sfm_rfe_19_2

['Frequency',
 'Recency',
 'Main_Product_mode_CIJ',
 'Lastest_Amt_Ratio',
 'Visit_per_TRX',
 'Avg_AMT']

## Tier 3

### Type 1 Model

In [46]:
#use all feature
rank3_18_all= best_Model(X3,y3,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X3,y3,pen,c,solver,hyper_param=True)
coef3_18_all,evaluation3_18_all=LogicResult_allcoef(X3,y3,best_pen,best_c,best_sol)
evaluation3_18_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.692593,0.719072,0.830357,0.770718


In [47]:
#10 cross validation result using best hyper parameter for all features
rank3_18_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=0.01 saga,0.702063,0.751956,0.829819,0.764997,l2,0.01,saga


In [48]:
#use selected feature
sfm_rfe_18_3=SFM_RFE_featrue_selection(X3,y3)
rank3_18= best_Model(X3[sfm_rfe_18_3],y3,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X3[sfm_rfe_18_3],y3,pen,c,solver,hyper_param=True)
coef3_18,evaluation3_18=LogicResult_allcoef(X3[sfm_rfe_18_3],y3,best_pen,best_c,best_sol)
evaluation3_18

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.690741,0.726542,0.806548,0.764457


In [49]:
#10 cross validation result using best hyper parameter for selected features
rank3_18.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=0.01 saga,0.707255,0.753009,0.838772,0.771437,l2,0.01,saga


### Type 2 Model

In [50]:
#use all feature
best_pen,best_c,best_sol = best_Model(X3,y3,pen,c,solver,hyper_param=True)
coef3_18_next_all,evaluation3_18_next_all=LogicResult_allcoef_next_year(X3,y3,X3_19\
                                                      ,y3_19,best_pen,best_c,best_sol)
evaluation3_18_next_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.757126,0.751295,0.885496,0.812894


In [51]:
#use selected feature
sfm_rfe_18_3_next=SFM_RFE_featrue_selection(X3,y3,fit_all=True)
best_pen,best_c,best_sol = best_Model(X3[sfm_rfe_18_3_next],y3,pen,c,solver,hyper_param=True)
coef3_18_next,evaluation3_18_next=LogicResult_allcoef_next_year(X3[sfm_rfe_18_3_next],y3,X3_19[sfm_rfe_18_3_next]\
                                                      ,y3_19,best_pen,best_c,best_sol)
evaluation3_18_next

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.755609,0.747755,0.890076,0.812732


### Type 3 Model

In [52]:
#use all feature
rank3_19_all= best_Model(X3_19,y3_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X3_19,y3_19,pen,c,solver,hyper_param=True)
coef3_19_all,evaluation3_19_all=LogicResult_allcoef(X3_19,y3_19,best_pen,best_c,best_sol)
evaluation3_19_all

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.745455,0.762238,0.832061,0.79562


In [53]:
#10 cross validation result using best hyper parameter for all features
rank3_19_all.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=0.01 saga,0.740162,0.793575,0.817735,0.779527,l2,0.01,saga


In [54]:
#use selected feature
sfm_rfe_19_3=SFM_RFE_featrue_selection(X3_19,y3_19)
rank3_19= best_Model(X3_19[sfm_rfe_19_3],y3_19,pen,c,solver)
best_pen,best_c,best_sol = best_Model(X3_19[sfm_rfe_19_3],y3_19,pen,c,solver,hyper_param=True)
coef3_19,evaluation3_19=LogicResult_allcoef(X3_19[sfm_rfe_19_3],y3_19,best_pen,best_c,best_sol)
evaluation3_19

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.75303,0.763761,0.847328,0.803378


In [55]:
#10 cross validation result using best hyper parameter for selected features
rank3_19.head(1)

,accuracy,precision,recall,f1,penalty,c,solver
Logistic l2 C=0.01 saga,0.751387,0.799654,0.828962,0.792391,l2,0.01,saga


In [56]:
#feature selected by Type 1 model for Tier 3
sfm_rfe_18_3

['Frequency',
 'Recency',
 'CUSTOMER_CLASS_DISTRIBUTOR',
 'CUSTOMER_CLASS_OEM',
 'Avg_AMT',
 'Avg_Duration']

In [57]:
#feature selected by Type 2 model for Tier 3
sfm_rfe_18_3_next

Index(['Avg_AMT', 'Avg_Duration', 'CUSTOMER_CLASS_OEM', 'Frequency',
       'Main_Product_mode_CIJ', 'Recency'],
      dtype='object')

In [58]:
#feature selected by Type 3 model for Tier 3
sfm_rfe_19_3

['Frequency',
 'Recency',
 'CUSTOMER_CLASS_OEM',
 'Visit_per_TRX',
 'Avg_AMT',
 'Avg_Duration']

# Churn List prediction for next year (Model Implementation)

In [111]:
def LogicResult_churn_next_year(X_cur,y_cur,X_next,Penalty,c,Solver,table=False):
    #use previous period to build model
    #use all data aviable to build feature and feed into model
    scaler_current=StandardScaler()
    scaler_current.fit(X_cur)
    X_cur_scaled = pd.DataFrame(scaler_current.transform(X_cur),columns = X_cur.columns)
    
    X_next_feature=X_next.drop('CUSTOMER_SITE_ID',axis=1)
    scaler_next=StandardScaler()
    scaler_next.fit(X_next_feature)
    X_next_feature_scaled = pd.DataFrame(scaler_next.transform(X_next_feature),columns = X_next_feature.columns)
    
    
    
    classifier = LogisticRegression(random_state=0, penalty=Penalty,solver=Solver,C=c)
    classifier.fit(X_cur_scaled, y_cur)
    
    coef_table = pd.DataFrame(list(X_cur.columns)).copy()
    coef_table.insert(len(coef_table.columns),"Coefs",classifier.coef_.transpose())
    coef_table.columns=['Variable','Coefs']
    coef_table['Coefs_ABS']=coef_table['Coefs'].abs()
    coef=coef_table.sort_values('Coefs_ABS',ascending=False).reset_index(drop=True)
    coef=coef.drop('Coefs_ABS',axis=1)
    
    y_pred = classifier.predict(X_next_feature_scaled)
    X_next['churn_pred'] = y_pred
    churn_next=X_next[X_next['churn_pred']==1]
    
    if table == False:
        return churn_next
    #if you want to view the feature chosen and coefficient
    else:
        return coef

## Tier 1

In [60]:
#using selected features and best hyper-parameter
sfm_rfe_19_1_next=SFM_RFE_featrue_selection(X1_19,y1_19,fit_all=True)
best_pen,best_c,best_sol = best_Model(X1_19[sfm_rfe_19_1_next],y1_19,pen,c,solver,hyper_param=True)
churn_20_1=LogicResult_churn_next_year(X1_19[sfm_rfe_19_1_next],y1_19,X1_20[['CUSTOMER_SITE_ID']+sfm_rfe_19_1_next]\
                                       ,best_pen,best_c,best_sol)

In [101]:
watchlist_1 =churn_20_1[churn_20_1['Recency']<366]

In [102]:
print(X1_20.shape[0])   #Total number of customer
print(churn_20_1.shape[0]) #Total number on the predicted inactive customer for next year
print(watchlist_1.shape[0]) #The customer that are active in the past year, but may churn in the following year

3391
218
4


In [108]:
watchlist_1

,CUSTOMER_SITE_ID,Recency,CUSTOMER_CLASS_OEM,Contract,Lastest_Amt_Ratio,Visit_per_TRX,churn_pred
705,9949,337,0,0.0,0.441129,0.071429,1
1051,24190,323,0,10.0,0.061347,0.220974,1
1965,63817,351,0,0.0,0.559203,0.100000,1
10858,112402,359,0,0.0,0.171816,0.000000,1


In [ ]:
#save the watchlist for Tier 1 as csv file
#watchlist_1.to_csv('Tier1_watchlist')

## Tier 2

In [64]:
#using selected features and best hyper-parameter
sfm_rfe_19_2_next=SFM_RFE_featrue_selection(X2_19,y2_19,fit_all=True)
best_pen,best_c,best_sol = best_Model(X2_19[sfm_rfe_19_2_next],y2_19,pen,c,solver,hyper_param=True)
churn_20_2=LogicResult_churn_next_year(X2_19[sfm_rfe_19_2_next],y2_19,X2_20[['CUSTOMER_SITE_ID']+sfm_rfe_19_2_next]\
                                       ,best_pen,best_c,best_sol)

In [103]:
watchlist_2 =churn_20_2[churn_20_2['Recency']<366]

In [104]:
print(X2_20.shape[0])   #Total number of customer
print(churn_20_2.shape[0]) #Total number on the predicted inactive customer for next year
print(watchlist_2.shape[0]) #The customer that are active in the past year, but may churn in the following year

5195
787
6


In [105]:
watchlist_2

,CUSTOMER_SITE_ID,Frequency,Recency,Lastest_Amt_Ratio,Visit_per_TRX,Avg_AMT,churn_pred
964,12910,7,252,1.406714,6.857143,14413.892857,1
1248,33594,80,318,0.378063,0.062500,105.304000,1
5136,367748,47,359,0.727133,0.574468,224.699574,1
5578,491145,102,33,0.970602,0.156863,46.791765,1
5849,534196,6,23,1.542149,6.333333,19449.466667,1
10577,70320,3,48,1.889250,0.666667,10082.570000,1


In [ ]:
#save the watchlist for Tier 2 as csv file
#watchlist_2.to_csv('Tier2_watchlist')

## Tier 3

In [68]:
sfm_rfe_19_3_next=SFM_RFE_featrue_selection(X3_19,y3_19,fit_all=True)
best_pen,best_c,best_sol = best_Model(X3_19[sfm_rfe_19_3_next],y3_19,pen,c,solver,hyper_param=True)
churn_20_3=LogicResult_churn_next_year(X3_19[sfm_rfe_19_3_next],y3_19,X3_20[['CUSTOMER_SITE_ID']+sfm_rfe_19_3_next]\
                                       ,best_pen,best_c,best_sol)

In [106]:
watchlist_3 =churn_20_3[churn_20_3['Recency']<366]

In [107]:
print(X3_20.shape[0])   #Total number of customer
print(churn_20_3.shape[0]) #Total number on the predicted inactive customer for next year
print(watchlist_3.shape[0]) #The customer that are active in the past year, but may churn in the following year

3810
2332
452


In [71]:
watchlist_3

,CUSTOMER_SITE_ID,Frequency,Recency,CUSTOMER_CLASS_OEM,NO_OF_SVC_INCIDENTS,Visit_per_TRX,Avg_AMT,Avg_Duration,churn_pred
87,1433,8,210,0,1,1.125000,167.102500,220.125000,1
148,2305,3,262,0,67,21.000000,117.413333,403.000000,1
231,3459,7,303,0,1,0.428571,166.321429,207.142857,1
317,4534,6,45,0,1,0.833333,457.335000,319.333333,1
383,5355,5,150,0,24,5.400000,665.498000,158.000000,1
...,...,...,...,...,...,...,...,...,...
14386,8397197,2,318,1,0,0.000000,357.890000,17.000000,1
14403,8398667,3,129,0,0,0.000000,3681.713333,81.666667,1
14492,8406846,2,93,0,0,0.000000,3167.160000,111.000000,1
14518,8408894,3,36,0,0,0.000000,3675.286667,16.333333,1


In [ ]:
#save the watchlist for Tier 3 as csv file
#watchlist_3.to_csv('Tier3_watchlist')

In [76]:
#basic stat for each tier based on most recent data
stat=second_df.groupby('Tier').agg({'Churn':'sum','CUSTOMER_SITE_ID':'count','Avg_Duration':'mean'}).reset_index()
stat['Churn_rate']=round((stat['Churn']/stat['CUSTOMER_SITE_ID'])*100,2)
stat

,Tier,Churn,CUSTOMER_SITE_ID,Avg_Duration,Churn_rate
0,Tier 1,257,3350,30.293444,7.67
1,Tier 2,1067,5019,83.728126,21.26
2,Tier 3,1965,3298,123.874558,59.58
3,Tier 4,1862,1864,1.301475,99.89
